In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from category_encoders import CountEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from category_encoders import CatBoostEncoder

In [ ]:
train = pd.read_csv("/kaggle/input/trains/train.csv")
test = pd.read_csv("/kaggle/input/trains/test.csv")

In [ ]:
#test = posta.tail(50000).copy()
#train = posta.head(190000).copy()
#test.drop(columns = ['precio'], inplace = True)

In [ ]:
dfCheto = pd.concat([train,test], sort=False)
dfCheto['precio'] = SimpleImputer(strategy='median').fit_transform(dfCheto[['precio']])
df = dfCheto.copy()

In [ ]:
df.provincia.fillna('provinciaVacia', inplace = True)
aux2 = df[['ciudad', 'provincia']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby(['provincia']).agg(lambda x:x.provincia.value_counts().index[0]).reset_index().set_index('ciudad')
ciuds = aux2.T.to_dict('records').copy()
ciuds = ciuds[0]
df.ciudad.fillna(df.provincia.map(ciuds), inplace = True)
df.ciudad.fillna('ciudadVacia', inplace=True)

In [ ]:
df.tipodepropiedad.fillna('tipoVacio', inplace=True)

In [ ]:
aux2 = df[['idzona', 'ciudad']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby('ciudad').agg({'idzona':'median'})
ids = aux2.T.to_dict('records').copy()
ids = ids[0]
df.idzona.fillna(df.ciudad.map(ids), inplace = True)
df['idzona'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['idzona']])

In [ ]:
aux = df[['lat', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lat':'mean'})
lats = aux.T.to_dict('records').copy()
lats = lats[0]
df.lat.fillna(df.ciudad.map(lats), inplace = True)
df['lat'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lat']])

In [ ]:
aux = df[['lng', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lng':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.lng.fillna(df.ciudad.map(lngs), inplace = True)
df['lng'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lng']])

In [ ]:
aux = df[['tipodepropiedad', 'habitaciones']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'habitaciones':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.habitaciones.fillna(df.tipodepropiedad.map(habts), inplace = True)
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])

In [ ]:
aux = df[['tipodepropiedad', 'garages']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'garages':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.garages.fillna(df.tipodepropiedad.map(habts), inplace = True)
df.garages.fillna(0, inplace=True)

In [ ]:
df.drop(columns=['direccion'], inplace=True)

In [ ]:
aux = df[['antiguedad', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'antiguedad':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.antiguedad.fillna(df.ciudad.map(lngs), inplace = True)
df['antiguedad'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['antiguedad']])

In [ ]:
aux = df[['banos', 'tipodepropiedad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'banos':'median'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.banos.fillna(df.tipodepropiedad.map(lngs), inplace = True)
df.banos.fillna(1, inplace=True)

In [ ]:
df.titulo.fillna('', inplace=True)
df.descripcion.fillna('', inplace=True)
df['texto'] = df.titulo + ' ' + df.descripcion
df['texto'] = df.texto.str.len()
df.drop(columns = ['descripcion', 'titulo'], inplace = True)

In [ ]:
aux = df.copy()
aux.metrostotales.fillna(0, inplace=True)
aux.metroscubiertos.fillna(0, inplace =True)
aux['metrostotalesCorregidos'] = aux[['metrostotales', 'metroscubiertos']].max(axis=1)
aux['metroscubiertosCorregidos'] = aux[['metrostotales', 'metroscubiertos']].min(axis=1)
df['metrostotales'] = aux['metrostotalesCorregidos']
df['metroscubiertos'] = aux['metroscubiertosCorregidos']

In [ ]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['dia'] = df.fecha.dt.day
df['mes'] = df.fecha.dt.month
df['anio'] = df.fecha.dt.year
df.drop(columns=['fecha'], inplace=True)

In [ ]:
df['publicacionesPorCiudad'] = CountEncoder().fit_transform(df['ciudad'])

In [ ]:
df['publicacionesPorProvincia'] = CountEncoder().fit_transform(df['provincia'])

In [ ]:
df['publicacionesPorTipo'] = CountEncoder().fit_transform(df['tipodepropiedad'])

In [ ]:
aux = df[['mes', 'anio']].copy()
aux['cantidad'] =  1
aux['cantidad'] = aux.groupby(['mes', 'anio']).transform('sum')
df['publicacionesEnMes'] = aux['cantidad']

In [ ]:
aux = df[['metrostotales', 'tipodepropiedad']].copy()
aux['metrosPromedioPorTipo'] = aux.groupby('tipodepropiedad').transform('mean')
df['metrosPromedioPorTipo'] = aux['metrosPromedioPorTipo']

In [ ]:
aux = df[['metrostotales', 'ciudad']].copy()
aux['metrosPromedioPorCiudad'] = aux.groupby('ciudad').transform('mean')
df['metrosPromedioPorCiudad'] = aux['metrosPromedioPorCiudad']

In [ ]:
transformador_categorias = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

transfomador_numero = Pipeline(steps=[('a', SimpleImputer(strategy = 'median'))])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero, ['id', 'antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'texto',
       'dia', 'mes', 'anio','publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad']),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])

In [ ]:
model = xgb.XGBRegressor(learning_rate= 0.1,n_estimators= 2000,scale_pos_weight= 2,max_depth= 5,min_child_weight= 5,gamma= 0.0,colsample_bytree= 0.75,
                     subsample= 0.7,colsample_bylevel= 0.65, tree_method='gpu_hist', gpu_id=0)

In [ ]:
entrenamiento = df.head(240000).copy()
prueba = df.tail(60000).copy()

In [ ]:
prueba.drop(columns=['precio'], inplace=True)

In [ ]:
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()

In [ ]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [ ]:
my_pipeline.fit(train_x, train_y)

In [ ]:
preds = my_pipeline.predict(train_x)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(preds, train_y)

In [ ]:
#precio = posta.precio.tail(60000).copy()

In [ ]:
preds = my_pipeline.predict(prueba)

In [ ]:
#mean_absolute_error(preds, precio)

Sin usar el precio obtengo un error de 570777 para el de test y de 499376 para el de train

In [ ]:
aux = df.tail(60000).copy()
aux.precio = preds
df = pd.concat([df.head(240000),aux], sort=False)

In [ ]:
df['catBoostCiudad'] = CatBoostEncoder().fit_transform(df[['ciudad']], df['precio'])
df['catBoostTipo'] = CatBoostEncoder().fit_transform(df[['tipodepropiedad']], df['precio'])
df['catBoostProvincia'] = CatBoostEncoder().fit_transform(df[['provincia']], df['precio'])

In [ ]:
aux = df[['precio', 'ciudad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('ciudad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorCiudad'] = df.ciudad.map(precioPromedio)

In [ ]:
aux = df[['precio', 'tipodepropiedad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('tipodepropiedad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorTipo'] = df.tipodepropiedad.map(precioPromedio)

In [ ]:
aux = df[['precio', 'idzona', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('idzona').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorZona'] = df.idzona.map(precioPromedio)

In [ ]:
aux = df[['precio', 'mes', 'anio', 'metrostotales']].copy()
aux['mes-anio'] = aux.mes + aux.anio
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('mes-anio').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['mes-anio'] = df.mes + df.anio
df['precioPromedioM2PorMes'] = df['mes-anio'].map(precioPromedio)

In [ ]:
df.drop(columns=['mes-anio'], inplace = True)

In [ ]:
entrenamiento = df.head(240000).copy()
prueba = df.tail(60000).copy()
prueba.drop(columns=['precio'], inplace=True)
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()

In [ ]:
df.columns

In [ ]:
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero, ['id', 'antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'texto',
       'dia', 'mes', 'anio','publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad', 'catBoostCiudad', 'catBoostTipo',
       'catBoostProvincia', 'precioPromedioM2PorCiudad',
       'precioPromedioM2PorTipo', 'precioPromedioM2PorZona',
       'precioPromedioM2PorMes']),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])

In [ ]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor2),
                              ('model', model)
                             ])

In [ ]:
my_pipeline.fit(train_x, train_y)

In [ ]:
preds = my_pipeline.predict(train_x)

In [ ]:
mean_absolute_error(preds, train_y)

In [ ]:
preds = my_pipeline.predict(prueba)

In [ ]:
#mean_absolute_error(preds, precio)

In [ ]:
res = pd.DataFrame(preds, index=prueba.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# RMSLE=1.0249284784393988 ?


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(res)